<a href="https://colab.research.google.com/github/LMDuarte/Data-Visualisation-Main-Page/blob/main/Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pushshift Module to extract Submissions Data from Reddit via Python

PRAW is pretty good at gettin reddit data but there are some limitations with it.
Including the removal of the [subreddit.submissions endpoint](https://www.reddit.com/r/changelog/comments/7tus5f/update_to_search_api/.). 

So for extracting Reddit submissions and the primarily data such as upvotes and comments count, I put together this notebook using Pushshift.

If you still prefer PRAW for extract submissions, I have written a code [template here](https://github.com/SeyiAgboola/Seyi_Projects/blob/master/submission_list.py).

I will also [host the code on GitHub](https://github.com/SeyiAgboola/Reddit-Data-Mining/blob/master/Using_Pushshift_Module_to_extract_Submissions.ipynb).

More info on the removal of the [subreddit.submissions endpoint](https://www.reddit.com/r/redditdev/comments/8bia9n/praw_psa_the_subredditsubmissions_method_no/).

# Import modules

In [ ]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

# Pushshift URL Examples

In [ ]:
#We can access the Pushshift API through building an URL with the relevant parameters without even needing Reddit credentials.
#These are some examples. You can follow the links and they will generate a page with JSON data
search_ps4_after_date = "https://api.pushshift.io/reddit/search/submission/?q=screenshot&after=1514764800&before=1517443200&subreddit=PS4"
search_science = "https://api.pushshift.io/reddit/search/submission/?q=science"

# Parameters for your Pushshift URL
These are probably the most important parameters to consider when building your Pushshift URL:

* size — increase limit of returned entries to 1000
* after — where to start the search
* before — where to end the search
* title — to search only within the submission’s title
* subreddit — to narrow it down to a particular subreddit

In [ ]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

# Extract key information from Submissions

We want key data for further analysis including: 
* Submission Title
* URL 
* Flair
* Author
* Submission post ID
* Score
* Upload Time
* No. of Comments 
* Permalink.


In [ ]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

# Update your Search Settings here

In [ ]:
#Create timestamps 

after = "1611928800" 
before = "1612145604" 
query = "gme" #Keyword(s) to look for in submissions
sub = "wallstreetbets" #Which Subreddit 

#subCount 
subCount = 0
#subStats 
subStats = {}

In [ ]:

data = getPushshiftData(query, after, before, sub)

while len(data) > 0: 
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611928800&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:08:00
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611929280&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:14:30
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611929670&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:20:51
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611930051&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:26:26
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611930386&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:33:45
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000&after=1611930825&before=1612145604&subreddit=wallstreetbets
100
2021-01-29 14:37:39
https://api.pushshift.io/reddit/search/submission/?title=gme&size=1000

JSONDecodeError: ignored

# Check your Submission Extraction was successful

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

23398 submissions have added to list
1st entry is:
Where to buy GME stocks in EU to help this cause? created: 2021-01-29 14:00:11
Last entry is:
DO NOT BE SWAYED BY THIS! HODL ON $GME 💎💎🚀💎💎 created: 2021-01-31 23:08:22


# Save data to CSV file

In [ ]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv
t31.csv
23398 submissions have been uploaded
